In [9]:
from ipyleaflet import Map, basemaps, CircleMarker
from ipywidgets import HTML
import pandas as pd
from data.labeled.preprocessed import LABELED_CITIES as cities, RISKS_MAPPING as risks
import numpy as np
from ipyleaflet import Icon, Marker, MarkerCluster, Popup

# First is latitude and second is longitude; both in degrees


m = Map(interpolation='nearest', basemap=basemaps.Stamen.Terrain,world_copy_jump=True)

markers = ()


popup = Popup(
            name='popup',
            close_button=False,
            auto_close=False,
            close_on_escape_key=False,auto_pan=False
        )
def mouseover_callback(marker,location, html):
    def callback(*args, **kwargs):
        popup.child = html
        popup.location = location
        m.add_layer(popup)
    return callback

def mouseout_callback():
    def callback(*args, **kwargs):
        m.remove_layer(popup)
    return callback



def populate_per_city(city, country, risks_dict):
    html =  f"""
    <p> <h4><b> {city}, {country} </b></h4>
    
    """
    
    for risk in risks:
        if risk not in risks_dict:
            continue
        risk_value = risks_dict[risk]
        if not pd.isnull(risk_value) and risk_value !=0:
            html += f"""
      <h4><b>{risks[risk]}</b>:{risk_value}</h4>
   
    """
    html += '</p>'
    return HTML(html)
    

for _,city in cities.iterrows():
    location=(city['latitude'],city['longitude'])
    html = populate_per_city(city.city, city.country, city)
    marker = CircleMarker(location=location,fill_color = "blue",color='blue',radius=2)
    marker.on_mouseover(mouseover_callback(marker,location, html))
    marker.on_mouseout(mouseout_callback())
    markers = markers + (marker,)
    
from utils.geo import  is_close

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:

from classification.model_handler import ModelHandler, InvalidCoordinates
from data.labeled.preprocessed import RISKS_MAPPING

handler = ModelHandler()

if not handler.is_fitted:
    handler.train()
    
risks_ids = sorted(RISKS_MAPPING)


manually_added_markers = []
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        try:
            output,mask = handler.test(coords[0], coords[1])
        except InvalidCoordinates:
            return
        print(output)
        html = populate_per_city("Close To:" + output['city'], output['country'],{k:v for k,v in output.items() if k in risks_ids})
        marker = CircleMarker(location=coords,fill_color = "red",color='red',radius=2)
        check = [is_close(marker.location,mark.location) for mark in manually_added_markers]
        if not any(check):
            
            marker.on_mouseover(mouseover_callback(marker,coords, html))
            marker.on_mouseout(mouseout_callback())
            manually_added_markers.append(marker)
            m.add_layer(marker)
        else:
            to_rem = [cnt for cnt,(m,c) in enumerate(zip(manually_added_markers,check)) if c][0]
            m.remove_layer(manually_added_markers[to_rem])
            manually_added_markers.pop(to_rem)
            m.remove_layer(popup)

m.on_interaction(handle_click)
m.add_layer(MarkerCluster(markers = markers))

In [12]:
m.layout.width = '80%'
m.layout.height = '1000px'
m.zoom=2.2
m.center = (0, 0)
display(m)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

hi
Gidan Maiaki
Nigeria
NG
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
{'risk0': 0.0, 'risk1': 0.0, 'risk2': 0.0, 'risk3': 0.0, 'risk4': 0.0, 'risk5': 0.0, 'risk6': 0.0, 'city': 'Gidan Maiaki', 'country': 'Nigeria'}
risk0          0.0
risk1          0.0
risk2          2.0
risk3          2.0
risk4          0.0
risk5          0.0
risk6          2.0
city         Abuja
country    Nigeria
Name: 2, dtype: object
risk0          0.0
risk1          0.0
risk2          2.0
risk3          2.0
risk4          0.0
risk5          0.0
risk6          2.0
city         Abuja
country    Nigeria
Name: 2, dtype: object


LayerException: layer not on map: Popup(auto_close=False, auto_pan=False, auto_pan_padding=[5, 5], child=HTML(value='\n    <p> <h4><b> Close To:Abuja, Nigeria </b></h4>\n    \n    \n      <h4><b>Inadequate or aging infrastructure</b>:2.0</h4>\n   \n    \n      <h4><b>Increased water stress or scarcity</b>:2.0</h4>\n   \n    \n      <h4><b>Higher water prices</b>:2.0</h4>\n   \n    </p>'), close_button=False, close_on_escape_key=False, location=[9.047287674516332, 7.324673112357872], name='popup', options=['auto_close', 'auto_pan', 'auto_pan_padding', 'auto_pan_padding_bottom_right', 'auto_pan_padding_top_left', 'close_button', 'close_on_escape_key', 'keep_in_view', 'max_height', 'max_width', 'min_width'])

risk0          0.0
risk1          0.0
risk2          2.0
risk3          2.0
risk4          0.0
risk5          0.0
risk6          2.0
city         Abuja
country    Nigeria
Name: 2, dtype: object
risk0          0.0
risk1          0.0
risk2          2.0
risk3          2.0
risk4          0.0
risk5          0.0
risk6          2.0
city         Abuja
country    Nigeria
Name: 2, dtype: object


LayerException: layer not on map: Popup(auto_close=False, auto_pan=False, auto_pan_padding=[5, 5], child=HTML(value='\n    <p> <h4><b> Close To:Abuja, Nigeria </b></h4>\n    \n    \n      <h4><b>Inadequate or aging infrastructure</b>:2.0</h4>\n   \n    \n      <h4><b>Increased water stress or scarcity</b>:2.0</h4>\n   \n    \n      <h4><b>Higher water prices</b>:2.0</h4>\n   \n    </p>'), close_button=False, close_on_escape_key=False, location=[9.039122034815534, 7.438635090050917], name='popup', options=['auto_close', 'auto_pan', 'auto_pan_padding', 'auto_pan_padding_bottom_right', 'auto_pan_padding_top_left', 'close_button', 'close_on_escape_key', 'keep_in_view', 'max_height', 'max_width', 'min_width'])

risk0          0.0
risk1          0.0
risk2          2.0
risk3          2.0
risk4          0.0
risk5          0.0
risk6          2.0
city         Abuja
country    Nigeria
Name: 2, dtype: object
hi
Arusu
Nigeria
NG
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
{'risk0': 0.0, 'risk1': 0.0, 'risk2': 0.0, 'risk3': 0.0, 'risk4': 0.0, 'risk5': 0.0, 'risk6': 0.0, 'city': 'Arusu', 'country': 'Nigeria'}
hi
Bakin Kog
Nigeria
NG
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
{'risk0': 0.0, 'risk1': 0.0, 'risk2': 0.0, 'risk3': 0.0, 'risk4': 0.0, 'risk5': 0.0, 'risk6': 0.0, 'city': 'Bakin Kog', 'country': 'Nigeria'}


LayerException: layer not on map: Popup(auto_close=False, auto_pan=False, auto_pan_padding=[5, 5], child=HTML(value='\n    <p> <h4><b> Close To:Arusu, Nigeria </b></h4>\n    \n    </p>'), close_button=False, close_on_escape_key=False, location=[8.68904518139311, 8.15964711507546], name='popup', options=['auto_close', 'auto_pan', 'auto_pan_padding', 'auto_pan_padding_bottom_right', 'auto_pan_padding_top_left', 'close_button', 'close_on_escape_key', 'keep_in_view', 'max_height', 'max_width', 'min_width'])

hi
Bokoko
Nigeria
NG
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
(1, 35)
{'risk0': 0.0, 'risk1': 0.0, 'risk2': 0.0, 'risk3': 0.0, 'risk4': 0.0, 'risk5': 0.0, 'risk6': 0.0, 'city': 'Bokoko', 'country': 'Nigeria'}
